# Project D12
## ADL1 - Cleaning Data

##### Opening the datasets 

In [157]:
import pandas as pd
import numpy as np
positions = pd.read_table("data/out.csv",sep = ",")
autonomy = pd.read_table("data/out_autonomy.csv",sep = ",")
speed = pd.read_table("data/out_speed.csv",sep = ",")
steer = pd.read_table("data/out_steer.csv",sep = ",")
turn = pd.read_csv("data/outturn.csv",sep = ",")

##### Convert data into pandas DataFrames

In [158]:
pos_df = pd.DataFrame(positions)
auto_df = pd.DataFrame(autonomy)
speed_df = pd.DataFrame(speed)
steer_df = pd.DataFrame(steer)
turn_df = pd.DataFrame(turn)

##### Deleting the beginning of the recording

In [159]:
def deletebeginning(pos_df):
    start_x = pos_df["field.pose.position.x"][0].round()#fixing the starting values
    print(start_x)
    start_y = pos_df["field.pose.position.y"][0].round()
    print(start_y)
    move_start_x=0#counters 
    move_start_y=0
    moved_x= False#value to represent whether the car has moved
    moved_y= False
    moved_index= 0#index counter
    for i in range(len(pos_df)):
        current_x=pos_df["field.pose.position.x"][i].round()#fixing the current x and y values
        current_y=pos_df["field.pose.position.y"][i].round()
        if current_x == start_x:# if the values do not change, they must be deleted
            move_start_x += 1
        else:
            moved_x= True
        if current_y == start_y:
            move_start_y += 1
        else:
            moved_y= True
            
        if moved_x and moved_y:# if the x and y values did change
            if move_start_x > move_start_y:#finding out which value changed last
                moved_index= move_start_x
            elif move_start_x < move_start_y:
                moved_index= move_start_y
    removable_indicies = pos_df[:moved_index]
    for index in range(len(removable_indicies)):
        remove_closest_time(speed_df, pos_df, index)
        remove_closest_time(steer_df, pos_df, index)
        remove_closest_time(turn_df, pos_df, index)
        pos_df.drop(index, 0, inplace=True)
        pos_df.reset_index(drop=True, inplace=True)
    print(moved_index)
    return pos_df

In [160]:
def deleteend(pos_df):
    start_x = pos_df["field.pose.position.x"][len(pos_df["field.pose.position.x"])-1].round()
    start_y = pos_df["field.pose.position.y"][len(pos_df["field.pose.position.y"])-1].round()
    stop_start_x=0
    stop_start_y=0
    stopped_x= False
    stopped_y= False
    stopped_index= 0
    print(start_x)
    print(start_y)
    i = len(pos_df["field.pose.position.x"])-1
    while i > 0:
        current_x=pos_df["field.pose.position.x"][i].round()
        current_y=pos_df["field.pose.position.y"][i].round()
        if current_x == start_x:
            stop_start_x += 1
        else:
            stopped_x= True
        if current_y == start_y:
            stop_start_y += 1
        else:
            stopped_y= True
            
        if stopped_x and stopped_y:
            if stop_start_x > stop_start_y:
                stopped_index= stop_start_x
            elif stop_start_x < stop_start_y:
                stopped_index= stop_start_y
        i -= 1
    
    real_stopped_index = len(pos_df["field.pose.position.x"])-stopped_index
    removable_indicies = pos_df[real_stopped_index:]
    for index in range(len(pos_df["field.pose.position.x"])-1, real_stopped_index-1, -1):
        remove_closest_time(speed_df, pos_df, index)
        remove_closest_time(steer_df, pos_df, index)
        remove_closest_time(turn_df, pos_df, index)
        pos_df.drop(index, 0, inplace=True)
        pos_df.reset_index(drop=True, inplace=True)
        
    print(real_stopped_index)
    return pos_df

In [161]:
pos_df = deletebeginning(pos_df)

-8354.0
-5678.0
111


In [162]:
pos_df = deleteend(pos_df)

-11672.0
-3078.0
54772


In [ ]:
pos_df

##### Finding the dataset with the best timestamp frequency

In [167]:
#pos_df: 20000003.229291935
#speed_df: 33486280.14943612
#steer_df: 33485068.742158055
#turn_df: 33486269.028086815
results = [20000003.229291935, 33486280.14943612, 33485068.742158055, 33486269.028086815]
print(min(results))

20000003.229291935


##### Method to delete all the necessary rows from all of the tables

In [168]:
def remove_closest_time(remove_table, origin_table, removed_index):
    #remove_table is the table from where we remove the row
    #origin_table is the table, where the original timestamp is
    #removed_index is the index in the origin table which timestamp we wish to compare
    closest = 10000000000000
    index = 0
    best_index = 0
    freq = 20000003.229291935 # best frequency across the tables from pos_df
    origin_table_time = origin_table["field.header.stamp"][removed_index] # time we wish to compare to other tables
    for time in remove_table["field.header.stamp"]:
        if abs(origin_table_time - time) < closest: # finding new best time compared to original
            closest = abs(origin_table_time - time)
            best_index = index
        index += 1
    if (closest / freq) < 1: # check to see whether the removable time seems valid or is unrelated.
        remove_table.drop(best_index, 0, inplace=True) # removes row
        remove_table.reset_index(drop=True, inplace=True) # resets indecies

##### Method to delete rows where the turn signal is active

In [182]:
def deleteturnsignalon():
    timestamptobedeleted = []
    for i in range(len(turn_df["field.enabled"])):
        if turn_df["field.enabled"][i] == 1:
            timestamptobedeleted.append(i)
    for index in timestamptobedeleted:
        remove_closest_time(pos_df,turn_df,index)
        remove_closest_time(speed_df,turn_df,index)
        remove_closest_time(steer_df,turn_df,index)
        turn_df.drop(index, 0, inplace=True)
        turn_df.reset_index(drop=True, inplace=True)

##### Method to delete rows where the car is reversing

In [184]:
def deletereversing(reversing_table): # reversing table is the table where the gear topic is in
    indicies_to_be_deleted = []
    for i in range(len(reversing_table["header.current_gear"])): # Finds all the rows where reverse is in
        if reversing_table["header.current_gear"] == 2:
            indicies_to_be_deleted.append(i)
    for index in indicies_to_be_deleted: # deletes all the corresponding rows from other files if they exist and from the current one
        remove_closest_time(pos_df,reversing_table,index)
        remove_closest_time(speed_df,reversing_table,index)
        remove_closest_time(steer_df,reversing_table,index)
        reversing_table.drop(index, 0, inplace=True)
        reversing_table.reset_index(drop=True, inplace=True)

##### Method to separate the curves from the straights

In [ ]:
def separatecurves():
    turnvalue = 1
    curvelist = [][]
    curvecounter = 0
    curvepart = 0
    for i in range(len(pos_df)):
        for j in range(len(turn_df)):
            if pos_df["field.header.stamp"][i] < turn_df["field.header.stamp"][j] or pos_df["field.header.stamp"][i] == turn_df["field.header.stamp"][j]:
                turnvalue = turn_df["field.output"][j]
                if turn_df["field.output"][j] == turnvalue and turnvalue != 1:
                    curvelist[curvecounter][curvepart]= i
                    i += 1
                    curvepart += 1
                    
        curvecounter += 1
                        
                       